In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 14 10:46:53 2019

@author: thomas
"""

#MODULES
import os,sys
import numpy as np
import pandas as pd
import seaborn
import tensorflow as tf
print(tf.__version__)
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()
import matplotlib.pyplot as plt
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 1.0e-2
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001

#Lists
#RLength
R = ["3","5","6","7"]
SwimDirList = ["SSL", "LSL", "Stat"]

allData = []


1.4.0


AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [ ]:
#Create a Model Class
class Model(object):
    def __init__(self):
        
        #Index of class names
        '''CHANGE'''
        self.class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']
        
        #Setup Neural Network model
        self.model = self.init_NN()
        
        #Setup the optimizer: Stochastic gradient descent (SGD)
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        
        #Setup Global Step
        self.global_step = tf.Variable(0)
        
        #Setup Loss and Accuracy Results
        self.train_loss_results = []
        self.train_accuracy_results = []
        
    def __call__(self,x):
        return self.W*x + self.b
    
    def init_NN(self):
        """NEURAL NETWORK MODEL"""
        #2 Hidden layers  with 10 nodes each and 1 output with 3 nodes (3 classifiers)
        #ReLU is common for hidden layer activation
        model = tf.keras.Sequential([
                tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)), #input shape
                tf.keras.layers.Dense(10, activation=tf.nn.relu),
                tf.keras.layers.Dense(3)
                ])
        return model
    
    #Define a Loss Function
    def loss(self, x, y):
        y_ = self.model(x)
        print(y_)
        #Takes the model's class probability predictions and the desired label
        #returns the average loss across the examples
        return tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
    
    def grad(self, inputs, targets):
        with tf.GradientTape() as tape:
            #print(tape.watched_variables())
            loss_value = self.loss(inputs, targets)
        return loss_value, tape.gradient(loss_value, self.model.trainable_variables)
    
    def train(self,train_dataset,num_epochs):
        """TRAINING LOOP"""
        #1) Iterate each epoch. An epoch is one pass through the dataset
        #2) Within an epoch, iterate over each example in the training Dataset
        #   grabbing its features (x) and label (y)
        #3) Using the example's features, make a prediction and compare it with the label
        #   Measure the inaccuracy of the prediction and use it to calculate the 
        #   model's loss and gradients
        #4) Use an optimizer to update the model's variables
        #5) Keep track of some stats for visualization
        #6) Repeat for each epoch
    
        ##Note: Rerunning this cell uses the same model variables
        
        for epoch in range(num_epochs):
            epoch_loss_avg = tfe.metrics.Mean()
            epoch_accuracy = tfe.metrics.Accuracy()
        
            #Training loop - using batches of 32
            for x, y in train_dataset:
                #Optimize the model
                loss_value, grads = self.grad(x, y)
                print('model variables')
                print(self.model.variables)
                return
                self.optimizer.apply_gradients(zip(grads, self.model.variables),self.global_step)
                
                #Track Progress
                epoch_loss_avg(loss_value) #add current batch loss
                #epoch_loss_avg = tf.metrics.mean(loss_value) #add current batch loss
                #compare predicted label to actual label
                epoch_accuracy(tf.argmax(self.model(x), axis=1, output_type=tf.int32),y)
                #epoch_accuracy = tf.metrics.accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32),y)
                
            #end epoch
            self.train_loss_results.append(epoch_loss_avg.result())
            self.train_accuracy_results.append(epoch_accuracy.result())
            
            if epoch % 50 == 0:
                print("Epoch {:03d}: Loss{:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                          epoch_loss_avg.result(),
                                                                          epoch_accuracy.result()))
        return
    
    def testModel(self,test_dataset):
        #Evaluate the Model
        #Uses only 1 epoch
        test_accuracy = tfe.metrics.Accuracy()
    
        for (x,y) in test_dataset:
            logits = self.model(x)
            prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
            test_accuracy(prediction, y)
        
        print("Test set accuracy: {:.3%}".format(test_accuracy.result()))
    
        tf.stack([y,prediction], axis=1)
        return
        
    
    def predict(self,predictions):
        for i, logits in enumerate(predictions):
            class_idx = tf.argmax(logits).numpy()
            p = tf.nn.softmax(logits)[class_idx]
            name = self.class_names[class_idx]
            print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))
        

def pack_features_vector(features, labels):
    """Pack the features into a single array"""
    features = tf.stack(list(features.values()),axis=1)
    return features,labels


In [ ]:
if __name__ == '__main__':
    #Download training dataset file
    train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"
    train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                               origin=train_dataset_url)
    
    print("Local copy of the dataset file: {}".format(train_dataset_fp))
    
    #column order in CSV file
    column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
                    'species']
    #features and labels
    feature_names = column_names[:-1]
    label_name = column_names[-1]
    
    print('Features: {}'.format(feature_names))
    print('Label: {}'.format(label_name))
    
    """TRAINING DATA"""
    #Create a tf.data.Dataset for training
    batch_size = 32
    train_dataset = tf.data.experimental.make_csv_dataset(
            train_dataset_fp,
            batch_size,
            column_names=column_names,
            label_name=label_name,
            num_epochs=1)
    
    features, labels = next(iter(train_dataset))
    
    print('Label: {}'.format(labels))
    
    #Use Seaborn to plot all pairwise plots
    #dataset = pd.read_csv(train_dataset_url, header=None, names=['sepal_length','sepal_width','petal_length','petal_width','species'])
    #print(dataset[1:])
    #seaborn.pairplot(dataset[1:], hue="species", size=2, diag_kind="kde")
    #plt.show()
    #return
    
    #Plot a few features from the batch
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    ax.scatter(features['petal_length'],
               features['sepal_length'],)
               #c=labels,
               #cmap='viridis')
    ax.set_xlabel('Petal length')
    ax.set_ylabel('Sepal length')
    
    train_dataset = train_dataset.map(pack_features_vector)
    #features element of dataset are now arrays with shape (batch_size, num_features)
    features, labels = next(iter(train_dataset))
    print(features[:5])
    
    """Create Model for Classification Learning"""
    model1 = Model()
    #The more hidden layers, the more powerful. BUT need more data in return
    #Predictions before Model is used
    predictions = model1.model(features)
    #print(model1.model.variables)
    #print non-normalized probability values
    print(predictions[:5])
    #print normalized probability values using softmax
    print(tf.nn.softmax(predictions[:5]))
    #print prediction using argmax
    #Note: should not be good predictions
    print("Prediction: {}".format(tf.argmax(predictions,axis=1)))
    print("    Labels: {}".format(labels))
    
    """TRAINING THE MODEL!!!"""
    l = model1.loss(features, labels)
    print("Loss Test: {}".format(l))
    #Calculate gradient tapes used to optimize our model
    #Calculate a single optimization step
    loss_value, grads = model1.grad(features, labels)
    print("Step: {}, Initial Loss: {}".format(model1.global_step.numpy(),
                                              loss_value.numpy()))
    model1.optimizer.apply_gradients(zip(grads, model1.model.variables),model1.global_step)
    print("Step: {},         Loss: {}".format(model1.global_step.numpy(),
                                              model1.loss(features,labels).numpy()))
    
    """TRAINING"""
    #Tunable variables
    num_epochs = 201
    model1.train(train_dataset, num_epochs)
    
    #Visualize Loss Function over Time
    fig, axes = plt.subplots(2, sharex=True, figsize=(12,8))
    fig.suptitle('Training Metrics')
        
    axes[0].set_ylabel("Loss",fontsize=14)
    axes[0].plot(model1.train_loss_results)
        
    axes[1].set_ylabel("Accuracy", fontsize=14)
    axes[1].set_xlabel("Epoch", fontsize = 14)
    axes[1].plot(model1.train_accuracy_results)
        
    """TEST THE MODEL!!!"""
    #Setup test dataset
    test_url = "http://download.tensorflow.org/data/iris_test.csv"
    test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                      origin=test_url)
    
    test_dataset = tf.contrib.data.make_csv_dataset(
            test_fp,
            batch_size,
            column_names=column_names,
            label_name='species',
            num_epochs=1,
            shuffle=False)
    
    test_dataset = test_dataset.map(pack_features_vector)
    
    #Evaluate the Model
    model1.testModel(test_dataset)
    
    #Make Predictions!
    predict_dataset = tf.convert_to_tensor([
            [5.1, 3.3, 1.7, 0.5],
            [5.9, 3.0, 4.2, 1.5],
            [6.9, 3.1, 5.4, 2.1]
            ])
    predictions = model1.model(predict_dataset)
    model1.predict(predictions)